# Equity of Induced Transit Trips

In this workbook we analyze how changes in travel time and demand brought on by the introduciton of additional transit infrastructure is distributed accross populations.

First, we set up our modules and load the appropriate data:

In [1]:
import os
import pandas as pd
import altair as alt
import numpy as np

# Replace with your data folder paths
mx_folder = r"data/matrices/input"
counts_folder = r"data/counts/input"
link_folder = r"data/counts/interim"
output_folder = r"data/counts/output"

As we load in our dataset, let's sum demand by each origin and display the total differences in transit trips made to learn about how much is induced:

In [2]:
dtypes = {'i':int, 'j':int, 'travel_time':float, 'demand':float}
mx_BAU = pd.read_csv(os.path.join(mx_folder, 'bau_times_flows.csv'), dtype=dtypes)[['i', 'demand']].groupby('i', as_index=False).sum()
mx_A = pd.read_csv(os.path.join(mx_folder, 'scenario_A_times_flows.csv'), dtype=dtypes)[['i', 'demand']].groupby('i', as_index=False).sum()
mx_B = pd.read_csv(os.path.join(mx_folder, 'scenario_B_times_flows.csv'), dtype=dtypes)[['i', 'demand']].groupby('i', as_index=False).sum()
mx_C = pd.read_csv(os.path.join(mx_folder, 'scenario_C_times_flows.csv'), dtype=dtypes)[['i', 'demand']].groupby('i', as_index=False).sum()

print("Scenario A:", (mx_A.demand.sum()-mx_BAU.demand.sum()), "extra demand")
print("Scenario B:", (mx_B.demand.sum()-mx_BAU.demand.sum()), "extra demand")
print("Scenario C:", (mx_C.demand.sum()-mx_BAU.demand.sum()), "extra demand")

Scenario A: 13733.152638852596 extra demand
Scenario B: 4025.5698905665195 extra demand
Scenario C: 2771.1263431145344 extra demand


Since we are using induced trips directly, we can join together our demand data for each scenario and calculate the deltas:

In [27]:
delta = pd.merge(mx_BAU[['i', 'demand']], mx_A[['i', 'demand']], on='i')
delta.columns = ['i', 'demand_BAU', 'demand_A']
delta = pd.merge(delta, mx_B[['i', 'demand']], on='i')
delta.columns = ['i', 'demand_BAU', 'demand_A', 'demand_B']
delta = pd.merge(delta, mx_C[['i', 'demand']], on='i')
delta.columns = ['i', 'demand_BAU', 'demand_A', 'demand_B', 'demand_C']

delta['delta_A'] = delta['demand_A'] - delta['demand_BAU']
delta['delta_B'] = delta['demand_B'] - delta['demand_BAU']
delta['delta_C'] = delta['demand_C'] - delta['demand_BAU']
delta.head()
print(delta.delta_A.sum())

# Write the file for mapping
delta.to_csv(os.path.join(output_folder, 'trips_added.csv'), index=False)
delta.head()

13733.1526388526


,i,demand_BAU,demand_A,demand_B,demand_C,delta_A,delta_B,delta_C
0,1,513.622080,480.545055,466.507274,472.622201,-33.077024,-47.114806,-40.999879
1,2,10.748341,8.560731,9.641028,9.808405,-2.187610,-1.107312,-0.939935
2,3,104.356331,96.522627,90.797636,86.584327,-7.833704,-13.558695,-17.772003
3,4,102.040002,240.352959,229.777685,230.856723,138.312957,127.737683,128.816722
4,5,28.774700,486.080989,391.578174,383.648842,457.306289,362.803474,354.874142


Finally, we can load in our demographic data and calculate the distributions of the score across various demographic groups

In [82]:
taz_da = pd.read_csv(os.path.join(link_folder, 'taz_da_link.csv'), dtype={'DAUID': int, 'taz_id': int, 'frac_da_in_taz': float})
da_demo = pd.read_csv(os.path.join(counts_folder, 'da_census_profile.csv'))
print(delta.delta_A.sum())

# Let's construct our TAZ to DA demographic link separately
taz_da_link = pd.merge(taz_da, da_demo, on='DAUID')
taz_da_link = pd.concat([taz_da_link[['taz_id']], taz_da_link[taz_da_link.columns[-10:-1]].multiply(taz_da_link['frac_da_in_taz'], axis="index")], axis=1)
taz_demo = taz_da_link.groupby('taz_id', as_index=False).sum()

delta_demo = pd.merge(taz_demo, delta, left_on='taz_id', right_on='i')

# Now let's assume that the proprotion of trips taken by a given group is proportional to their population makeup
# E.g. if there are a 100 net new trips in a zone with 60% visible minority population, we assume 60% of those trips were taken by visible minorities

delta_demo['A_vm_minority'] = (delta_demo['vm_minority']/delta_demo['vm_total'])*delta_demo['delta_A']
delta_demo['A_income_lim'] = (delta_demo['income_lim']/delta_demo['income_total'])*delta_demo['delta_A']
delta_demo['A_labour_unemployed'] = (delta_demo['labour_unemployed']/delta_demo['labour_total'])*delta_demo['delta_A']

delta_demo['B_vm_minority'] = (delta_demo['vm_minority']/delta_demo['vm_total'])*delta_demo['delta_B']
delta_demo['B_income_lim'] = (delta_demo['income_lim']/delta_demo['income_total'])*delta_demo['delta_B']
delta_demo['B_labour_unemployed'] = (delta_demo['labour_unemployed']/delta_demo['labour_total'])*delta_demo['delta_B']

delta_demo['C_vm_minority'] = (delta_demo['vm_minority']/delta_demo['vm_total'])*delta_demo['delta_C']
delta_demo['C_income_lim'] = (delta_demo['income_lim']/delta_demo['income_total'])*delta_demo['delta_C']
delta_demo['C_labour_unemployed'] = (delta_demo['labour_unemployed']/delta_demo['labour_total'])*delta_demo['delta_C']

# What we want to do now is get the total trips by a demographic and divide by the total population of that demographic
sums = delta_demo[[
    'A_vm_minority', 'B_vm_minority', 'C_vm_minority', 
    'A_income_lim', 'B_income_lim', 'C_income_lim', 
    'A_labour_unemployed', 'B_labour_unemployed', 'C_labour_unemployed',
    'vm_minority', 'income_lim', 'labour_unemployed'
]].sum()

# Finally we take ratios to get trips per 1,000 people.
to_plot = pd.DataFrame({
    'category': ['pop_2016', 'pop_2016', 'pop_2016', 'vm_minority', 'income_lim', 'labour_unemployed', 'vm_minority', 'income_lim', 'labour_unemployed', 'vm_minority', 'income_lim', 'labour_unemployed'],
    'value': [
        delta.delta_A.sum()/(delta_demo['pop_2016'].sum()/1000),
        delta.delta_B.sum()/(delta_demo['pop_2016'].sum()/1000),
        delta.delta_C.sum()/(delta_demo['pop_2016'].sum()/1000),
        sums['A_vm_minority']/(sums['vm_minority']/1000),
        sums['A_income_lim']/(sums['income_lim']/1000),
        sums['A_labour_unemployed']/(sums['labour_unemployed']/1000),
        sums['B_vm_minority']/(sums['vm_minority']/1000),
        sums['B_income_lim']/(sums['income_lim']/1000),
        sums['B_labour_unemployed']/(sums['labour_unemployed']/1000),
        sums['C_vm_minority']/(sums['vm_minority']/1000),
        sums['C_income_lim']/(sums['income_lim']/1000),
        sums['C_labour_unemployed']/(sums['labour_unemployed']/1000),
        ],
    'scenario': ['A', 'B', 'C', 'A', 'A', 'A', 'B', 'B', 'B', 'C', 'C', 'C']
    })
to_plot

13733.1526388526


,category,value,scenario
0,pop_2016,5.029686,A
1,pop_2016,1.474341,B
2,pop_2016,1.014909,C
3,vm_minority,5.558348,A
4,income_lim,4.684783,A
5,labour_unemployed,5.556481,A
6,vm_minority,1.474980,B
7,income_lim,1.250315,B
8,labour_unemployed,1.422330,B
9,vm_minority,0.861685,C


Finally, we assemble our data into a plottable form and generate plots.

In [83]:
pretty_names = {'income_lim': "Low Income (LIM)", 'pop_2016': 'Total Population', 'vm_minority': "Visible Minority", 'labour_unemployed':"Unemployed"}
to_plot['demo_name'] = to_plot.category.map(pretty_names)
to_plot.head()

,category,value,scenario,demo_name
0,pop_2016,5.029686,A,Total Population
1,pop_2016,1.474341,B,Total Population
2,pop_2016,1.014909,C,Total Population
3,vm_minority,5.558348,A,Visible Minority
4,income_lim,4.684783,A,Low Income (LIM)


In [88]:
alt.Chart(to_plot).mark_bar().encode(
    alt.Y('scenario:N', title=None),
    alt.X('value:Q', title='Trips per 1,000 Individuals'),
    alt.Color('scenario:N', title='Scenario'),
    alt.Row('demo_name:N', title='', sort=['Total Population'], spacing=35)
).properties(
    title="Average Additional Transit Demand for SmartTrack Scenarios",
    width=600,
    height=80
).configure(font='Roboto').configure_axis(grid=False).configure_view(strokeWidth=0).configure_title(fontSize=18)

alt.Chart(...)